# requirements
pandas, plotly and qgrid

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import datetime

from compute import *

In [2]:
#constants
ProjectName = "3751 Third Ave - 5 plex"

# assumptions
NumberUnits=5
price     = 1850000
rent      = 2500 + 1000 + 1300 + 1200 + 1400
Carry     = 0
ltv       = 70
loanRatePer = 3.5
expenseRate = 35
# Proforma
Description  = "Raise rents minimal renovation, charge for parking, storage, plus 25 water fee and laundry"
rentPF       = 2500+1700*3+1400+600+200+5*25+75
expenseRatePF= 32.5
capPF        = 4
repairCost   = 5000*4+5000
computeProjectSimple(ProjectName, NumberUnits, price, rent,expenseRate, Carry,ltv, loanRatePer, rentPF,expenseRatePF,capPF,repairCost,
                    description=Description)




        Project   : 3751 Third Ave - 5 plex
        Purchase  : $1,850,000
        rent roll : $88,800
        NOI       : $57,720  
        Expen Rate: 35%
        Cap Rate  : 3.12%        
        DebService:
           amount - $1,295,000
           rate   - 3.5%
           payment- 69,782$
        DSCR      : 0.83%  
        
    
        Descript  : Raise rents minimal renovation, charge for parking, storage, plus 25 water fee and laundry
        Basis     : $1,875,000
        Cap Rate  : 4.32%        
        rent roll : $120,000
        NOI       : $81,000
        Expen Rate: 32.5%    
        Price     : $2,025,000 @ cap(4.00%)
        


# construction stage Proposal-Front-4x3

currently a 3600 sqft building, convert 3bd/2ba--> to a 1bd1ba & 2bd1ba
then construct a four story building with an elevator, no parking! each flow will have two studios and one 1bd1ba

units sizes are
15x40 ==> 600
15x40 ==> 600
10x40+10x20 ==>  50


In [4]:
from IPython.display import display, HTML
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.offline import iplot
from ipywidgets import *

In [25]:
# new building
newSqft           = 7200
numNewUnits       = 12
numUnits          = 18
# [%] soft cost architect engineers
softBuildCost     = 5
#35k 3bd to 2X1bd1ba, 50k exterior and ~20 for each unit
oldBuildCost      = 35000+50000+100000 
# deck over existing build used by all tenants, over 1500
deck              = 100000
# concrete pedastool for first floor, to allow for future retail
concretePedaStool = 100000
# elevator
Elevator          = 100000
# util setup cost per unit
utilSetupPerUnit  = 25000
# [%] assuming purchase/construction cost and assuming all tenants vacant (assume linear withdrawl,hence divide by 2)
carryCostRate     = 5
# include points, appraisal, escrow title and 
carryCostPoints   = 3
purchasePrice     = 1850000
fig = go.Figure()   
df = pd.DataFrame(columns=["Valuation","BuildCost","CarryCost","TotalCost","Profit","ExpRate","rentRoll","Cap","CostPerSqft"],dtype=int)
idx = 0
for expRate in np.arange(30,35+1,2.5):
    for rent in np.arange(1500,2000+1,100):
        for capPF in np.arange(3,5+.1,.15):        
            for costPerSqft in np.arange(175,305+1,15):
                buildCost = newSqft * costPerSqft + oldBuildCost + deck + concretePedaStool + Elevator +\
                    utilSetupPerUnit  *numNewUnits
                buildCost = buildCost*(100+softBuildCost)/100.
                rentRoll  = numUnits * rent
                valuation = rentRoll * (100-expRate)/100. * 12 / (capPF/100.)
                carryCost = (purchasePrice+buildCost/2)*carryCostRate/100. + (purchasePrice+buildCost) *carryCostPoints/1000
                totalCost = purchasePrice + buildCost + carryCost                
                profit    = int(valuation-totalCost)
                df.loc[idx] = [int(valuation),int(buildCost),int(carryCost),int(totalCost),profit,expRate,rentRoll,capPF,costPerSqft]
                idx+=1


In [28]:
import plotly.express as px

#fig = go.Figure()
#fig.add_trace(go.Scatter(y=df["Valuation"],line = dict(color='green', width=1, dash='dash')))
#fig = px.scatter_3d(df,x="CostPerSqft",y="Cap",z="Valuation")
fig = px.parallel_coordinates(df, color="Profit",
                              dimensions=['CostPerSqft',
                                          'Cap', 
                                          #'ExpRate',
                                          'rentRoll',
                                          "Profit"
                                          ])
fig.show()

In [18]:
from IPython.display import display
from ipywidgets import widgets, interactive
import qgrid
qgrid.show_grid(df,
                grid_options={'fullWidthRows': True,
                              'syncColumnCellResize': True,
                              'forceFitColumns': True,
                              'rowHeight': 40,
                              'enableColumnReorder': True,
                              'enableTextSelectionOnCells': True,
                              'editable': True})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…